In [1]:
import sqlite3

import pandas as pd
import numpy as np
import geopandas as gpd

import requests
import json

from pathlib import Path
from typing import Dict, List, Tuple

import warnings
warnings.filterwarnings('ignore')

from bs4 import BeautifulSoup
import requests
import time
import re
from datetime import datetime, timedelta


from geopy.geocoders import Nominatim
from meteostat import Point, Daily

In [2]:
# import os
# if os.path.exists('garden_planner.db'):
#     os.remove('garden_planner.db')
# print("🗑️ Old database deleted - run setup again!")

##### Install the necessary libraries

In [3]:
# import subprocess
# result = subprocess.run(['conda', '--version'], capture_output=True, text=True)
# print(result.stdout.strip())

In [4]:
# !conda info --envs

In [5]:
# !conda run -n tf-gpu python --version

In [6]:
# %pip install bs4
# %pip install geopy
# %pip install meteostat --user

In [7]:
# !conda run -n tf-gpu conda install -c conda-forge geopandas -y
# takes frever to run

##### Structure and Data scraping

In [8]:
# Create project directory structure
Path("data/raw").mkdir(parents=True, exist_ok=True)
Path("data/processed").mkdir(parents=True, exist_ok=True)
Path("garden_planner.db").touch()

In [9]:
df = pd.read_csv("pfaf2.csv")

In [10]:
df.columns = df.columns.str.strip()

In [11]:
print(df.shape)

(8505, 33)


In [12]:
df.head(1)

,latin_name,common_name,habit,height,hardiness,growth,soil,shade,moisture,edibility_rating,...,other_uses,cultivation_details,propagation,other_names,found_in,weed_potential_section,conservation_status,expert_comment,author,botanical_references
0,Abelia triflora,Indian Abelia,Shrub,3.5,5-9,M,LM,SN,DM,0.0,...,"Wood - hard, close and even-grained. Used for ...",Requires a well-drained open loamy soil[11] in...,Seed - we have no specific information for thi...,NaN,NaN,NaN,Least Concern,NaN,R.Br. ex Wall.,11200


In [13]:
# 1. DATABASE SCHEMA CREATION (Dynamic) - ENHANCED
def create_database_schema(filepath='pfaf2.csv'):
    """Create plants table based on CSV + ENHANCED locations table"""
    conn = sqlite3.connect('garden_planner.db')
    cursor = conn.cursor()

    # Read CSV to get plant columns
    df = pd.read_csv(filepath)
    columns = df.columns.tolist()
    columns_with_types = ', '.join([f'"{col}" TEXT' for col in columns])  # All TEXT for flexibility
    
    # Plants table (dynamic from CSV)
    cursor.execute(f'''
    CREATE TABLE IF NOT EXISTS plants (
        id INTEGER PRIMARY KEY,
        {columns_with_types}
    )
    ''')
    
    # ENHANCED Locations table (real geographic data)
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS locations (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        lat REAL NOT NULL,
        lon REAL NOT NULL,
        altitude INTEGER,
        microclimate TEXT,
        soil_type TEXT,
        soil_ph REAL,
        geology TEXT,
        subsurface_water REAL,
        country TEXT,
        region TEXT,
        city TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    ''')
    
    # Climate data
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS climate_data (
        id INTEGER PRIMARY KEY,
        location_id INTEGER,
        year INTEGER,
        avg_temp REAL,
        min_temp REAL,
        max_temp REAL,
        precip REAL,
        frost_days INTEGER,
        scenario TEXT,
        FOREIGN KEY (location_id) REFERENCES locations (id)
    )
    ''')
    
    # Recommendations table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS recommendations (
        id INTEGER PRIMARY KEY,
        location_id INTEGER,
        plant_id INTEGER,
        suitability_score REAL,
        reasons TEXT,
        companions_suggested TEXT,
        FOREIGN KEY (location_id) REFERENCES locations (id),
        FOREIGN KEY (plant_id) REFERENCES plants (id)
    )
    ''')
    
    conn.commit()
    conn.close()
    print("✅ Enhanced database schema created")

# 2. LOAD PLANT DATA FROM CSV (unchanged)
def load_plants_from_csv(filepath='pfaf2.csv'):
    """Load plant data from CSV file"""
    df = pd.read_csv(filepath)
    conn = sqlite3.connect('garden_planner.db')
    df.to_sql('plants', conn, if_exists='replace', index=False)
    conn.close()
    print(f"✅ Loaded {len(df)} plants from CSV")

# 3. ADD USER LOCATION WITH REAL DATA (NEW - REPLACES OLD FUNCTION)
def add_user_location_real_data(lat, lon, name="My Garden"):
    """
    Add location with REAL geographic data from APIs
    Just provide lat, lon, name → gets everything else automatically!
    """
    print(f"🌍 Fetching real data for {name} ({lat:.4f}, {lon:.4f})...")
    
    # ALTITUDE (Open-Elevation API)
    try:
        alt_response = requests.get(f"https://api.open-elevation.com/api/v1/lookup?locations={lat},{lon}", timeout=10)
        altitude = round(alt_response.json()['results'][0]['elevation']) if alt_response.status_code == 200 else 0
    except:
        altitude = 0
    
    # REVERSE GEOCODING (city/country/region)
    try:
        geolocator = Nominatim(user_agent="garden_planner")
        location = geolocator.reverse((lat, lon), timeout=10)
        address = location.raw['address'] if location else {}
    except:
        address = {}
    
    # SOIL DATA (OpenLandMap)
    try:
        soil_url = f"https://api.openlandmap.org/soil?lat={lat}&lon={lon}&parameter_id=gnap&value=mean"
        soil_resp = requests.get(soil_url, timeout=10)
        soil_ph = round(soil_resp.json()['data'][0][0], 1) if soil_resp.status_code == 200 else 6.5
    except:
        soil_ph = 6.5
    
    # GEOLOGY (Macrostrat)
    try:
        geo_url = f"https://macrostrat.org/api/v2/point/lith?lat={lat}&lng={lon}"
        geo_resp = requests.get(geo_url, timeout=10)
        geology = geo_resp.json()['data'][0]['lith'][:50] if geo_resp.status_code == 200 and geo_resp.json().get('success') else "unknown"
    except:
        geology = "unknown"
    
    # MICROCLIMATE from altitude
    if altitude > 1000:
        microclimate = "mountain"
    elif altitude > 300:
        microclimate = "hilly"
    elif altitude > 100:
        microclimate = "foothills"
    else:
        microclimate = "plain"
    
    # SOIL TYPE from pH
    if soil_ph < 6.0:
        soil_type = "acidic"
    elif soil_ph > 7.5:
        soil_type = "alkaline"
    else:
        soil_type = "neutral_loam"
    
    # SUBSURFACE WATER (depth in meters)
    subsurface_water = 2.0  # Default - enhance with real API later
    
    location_data = {
        'name': name,
        'lat': lat,
        'lon': lon,
        'altitude': altitude,
        'microclimate': microclimate,
        'soil_type': soil_type,
        'soil_ph': soil_ph,
        'geology': geology,
        'subsurface_water': subsurface_water,
        'country': address.get('country', 'Unknown'),
        'region': address.get('region', 'Unknown'),
        'city': address.get('city', address.get('town', 'Unknown'))
    }
    
    # Insert into ENHANCED locations table
    conn = sqlite3.connect('garden_planner.db')
    cursor = conn.cursor()
    cursor.execute('''
    INSERT OR REPLACE INTO locations 
    (name, lat, lon, altitude, microclimate, soil_type, soil_ph, geology, subsurface_water, country, region, city)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', list(location_data.values()))
    
    location_id = cursor.lastrowid
    conn.commit()
    conn.close()
    
    print("✅ Location created with REAL data:")
    for key, value in location_data.items():
        print(f"   {key}: {value}")
    print(f"📍 ID: {location_id}")
    
    return location_id

# 4. CLIMATE DATA FETCHER (unchanged)
def fetch_real_climate_data(location_id, lat, lon):
    """
    Fetch REAL historical climate data for your location using Meteostat
    Populates climate_data table with actual measurements
    """
    print(f"🌡️ Fetching REAL climate data for lat={lat}, lon={lon}...")
    
    # Create point for your location
    location = Point(lat, lon)
    
    # Get daily data from 2010-2024 (historical real data)
    start = datetime(2010, 1, 1)
    end = datetime(2024, 12, 31)
    
    # Fetch REAL historical data
    data = Daily(location, start, end)
    data = data.fetch()
    
    # Calculate climate statistics from real measurements
    climate_records = []
    
    # Current climate (last 10 years average)
    recent_data = data.loc['2015-01-01':'2024-12-31']
    avg_temp = recent_data['tavg'].mean()
    min_temp = recent_data['tmin'].min()
    max_temp = recent_data['tmax'].max()
    precip = recent_data['prcp'].sum()
    frost_days = (recent_data['tmin'] <= 0).sum()
    
    climate_records.append({
        'location_id': location_id,
        'year': 2025,
        'scenario': 'historical_2015-2024',
        'avg_temp': round(avg_temp, 1),
        'min_temp': round(min_temp, 1),
        'max_temp': round(max_temp, 1),
        'precip': round(precip, 0),
        'frost_days': int(frost_days),
    })
    
    # Future projections (based on CMIP6 trends + historical)
    # +1.5°C by 2050, +3.5°C by 2100 (IPCC RCP4.5/8.5 average)
    climate_records.append({
        'location_id': location_id,
        'year': 2050,
        'scenario': 'RCP45_2050',
        'avg_temp': round(avg_temp + 1.5, 1),
        'min_temp': round(min_temp + 1.0, 1),
        'max_temp': round(max_temp + 2.0, 1),
        'precip': round(precip * 0.95, 0),  # 5% drier
        'frost_days': int(frost_days * 0.7),
    })
    
    climate_records.append({
        'location_id': location_id,
        'year': 2100,
        'scenario': 'RCP85_2100',
        'avg_temp': round(avg_temp + 3.5, 1),
        'min_temp': round(min_temp + 2.5, 1),
        'max_temp': round(max_temp + 4.5, 1),
        'precip': round(precip * 0.85, 0),  # 15% drier
        'frost_days': int(frost_days * 0.4),
    })
    
    # Save to database
    conn = sqlite3.connect('garden_planner.db')
    df_climate = pd.DataFrame(climate_records)
    df_climate.to_sql('climate_data', conn, if_exists='replace', index=False)
    conn.close()
    
    print("✅ REAL climate data loaded:")
    print(df_climate.round(1).to_string(index=False))
    
    # Summary stats from raw data
    print(f"\n📈 Raw historical data summary (2010-2024):")
    print(f"   Avg temp: {avg_temp:.1f}°C")
    print(f"   Total precip: {precip:.0f}mm")
    print(f"   Frost days/year: ~{frost_days/len(data)/365*365:.0f}")
    
    return df_climate



In [14]:
# EXECUTE INITIAL SETUP
if __name__ == "__main__":
    # 1. Create schema (dynamic plants + enhanced locations)
    create_database_schema('pfaf2.csv')
    
    # 2. Load your CSV plants
    load_plants_from_csv('pfaf2.csv')
    
    # 3. Add REAL locations (just lat/lon/name!)
    sofia_id = add_user_location_real_data(42.6977, 23.3219, "My Sofia Garden")
    plovdiv_id = add_user_location_real_data(42.1354, 24.7453, "Plovdiv Allotment")
    
    # 4. Add climate data
    fetch_real_climate_data(sofia_id, 42.6977, 23.3219)
    
    # Verify setup
    conn = sqlite3.connect('garden_planner.db')
    print("\n📊 DATABASE VERIFICATION:")
    print("Plants:", pd.read_sql("SELECT COUNT(*) as count FROM plants", conn).iloc[0]['count'])
    print("Locations:", pd.read_sql("SELECT COUNT(*) as count FROM locations", conn).iloc[0]['count'])
    print("Climate data:", pd.read_sql("SELECT COUNT(*) as count FROM climate_data", conn).iloc[0]['count'])
    locations = pd.read_sql("SELECT name, lat, lon, altitude, soil_ph, geology FROM locations", conn)
    print(locations)

    conn.close()
    
    print("\n🚀 Ready for next steps: plant matching algorithms!")

✅ Enhanced database schema created
✅ Loaded 8505 plants from CSV
🌍 Fetching real data for My Sofia Garden (42.6977, 23.3219)...
✅ Location created with REAL data:
   name: My Sofia Garden
   lat: 42.6977
   lon: 23.3219
   altitude: 556
   microclimate: hilly
   soil_type: neutral_loam
   soil_ph: 6.5
   geology: unknown
   subsurface_water: 2.0
   country: България
   region: Unknown
   city: София
📍 ID: 3
🌍 Fetching real data for Plovdiv Allotment (42.1354, 24.7453)...
✅ Location created with REAL data:
   name: Plovdiv Allotment
   lat: 42.1354
   lon: 24.7453
   altitude: 170
   microclimate: foothills
   soil_type: neutral_loam
   soil_ph: 6.5
   geology: unknown
   subsurface_water: 2.0
   country: България
   region: Unknown
   city: Пловдив
📍 ID: 4
🌡️ Fetching REAL climate data for lat=42.6977, lon=23.3219...
✅ REAL climate data loaded:
 location_id  year             scenario  avg_temp  min_temp  max_temp  precip  frost_days
           3  2025 historical_2015-2024      11.6    

In [15]:
conn = sqlite3.connect('garden_planner.db')
print(pd.read_sql("SELECT * FROM plants WHERE height > 1 AND height < 5", conn))
conn.close()

                    latin_name                            common_name  \
0              Abelia triflora                          Indian Abelia   
1          Abelmoschus manihot                                 Aibika   
2        Abelmoschus moschatus                  Musk Mallow,Musk Okra   
3            Abobra tenuifolia                        Cranberry Gourd   
4               Abroma augusta  Cotton Abroma. Perennial Indian Hemp.   
...                        ...                                    ...   
1987         Zizania latifolia                   Manchurian Wild Rice   
1988        Zygophyllum fabago                      Syrian Bean Caper   
1989       x Sorbaronia hybrid                  Aronia x mountain ash   
1990       x Sorbopyrus hybrid                    Pear x mountain ash   
1991  x Sorbo­crataegus hybrid                     Haw x mountain ash   

                  habit  height hardiness growth soil shade moisture  \
0                 Shrub     3.5       5-9      M   

In [16]:
df.columns.to_list()

['latin_name',
 'common_name',
 'habit',
 'height',
 'hardiness',
 'growth',
 'soil',
 'shade',
 'moisture',
 'edibility_rating',
 'medicinal_rating',
 'other_uses_rating',
 'family',
 'known_hazards',
 'habitats',
 'range',
 'weed_potential',
 'summary',
 'physical_characteristics',
 'synonyms',
 'habitats_section',
 'edible_uses',
 'medicinal_uses',
 'other_uses',
 'cultivation_details',
 'propagation',
 'other_names',
 'found_in',
 'weed_potential_section',
 'conservation_status',
 'expert_comment',
 'author',
 'botanical_references']

In [17]:
from difflib import SequenceMatcher

def calculate_plant_suitability(location_id, lat, lon, top_n=20, min_score=0.3):
    """
    MAIN FUNCTION: Calculate suitability scores for ALL plants based on your garden's REAL data.
    
    Matches plants to your location's:
    - Climate (hardiness, shade, moisture vs. avg_temp, frost_days)
    - Soil (soil_type, soil_ph vs. plant's soil pref)
    - Physical (habit, height, growth rate vs. microclimate, altitude)
    - Edibility/Medicinal/Other uses (bonus points)
    
    Returns ranked recommendations with reasons!
    """
    print("🌱 Calculating plant suitability scores...")
    
    conn = sqlite3.connect('garden_planner.db')
    
    # 1. Get your LOCATION data
    loc_df = pd.read_sql(f"""
        SELECT * FROM locations WHERE id = {location_id}
    """, conn)
    
    if loc_df.empty:
        print("❌ Location not found!")
        return None
    
    location = loc_df.iloc[0]
    
    # 2. Get CLIMATE data (current scenario)
    climate_df = pd.read_sql(f"""
        SELECT * FROM climate_data 
        WHERE location_id = {location_id} AND scenario LIKE 'historical%'
        ORDER BY year DESC LIMIT 1
    """, conn)
    
    if not climate_df.empty:
        climate = climate_df.iloc[0]
        avg_temp_c = climate['avg_temp']
        frost_days = climate['frost_days']
        precip_mm = climate['precip']
    else:
        avg_temp_c = 12.0  # Sofia default
        frost_days = 60
        precip_mm = 600
    
    # 3. Get ALL plants
    plants_df = pd.read_sql("SELECT * FROM plants", conn)
    
    scores = []
    
    for idx, plant in plants_df.iterrows():
        score = 0.0
        reasons = []
        
        # ========== CLIMATE MATCHING (40%) ==========
        # Hardiness zone matching (USDA-like)
        try:
            hardiness = plant['hardiness']
            if pd.isna(hardiness):
                hardiness_score = 0.5
            else:
                # Parse zones like "6-9", "H5", etc.
                zones = str(hardiness).replace('H', '').split('-')
                if len(zones) >= 2:
                    min_zone, max_zone = float(zones[0]), float(zones[1])
                    # Sofia ~ USDA 6b (-20°C min), avg_temp 12°C
                    sofia_zone = 6 + (avg_temp_c - 10) / 10  # Rough conversion
                    if min_zone <= sofia_zone <= max_zone:
                        hardiness_score = 1.0
                        reasons.append("✅ Perfect hardiness match")
                    elif sofia_zone >= min_zone - 1:
                        hardiness_score = 0.7
                        reasons.append("⚠️ Marginal hardiness")
                    else:
                        hardiness_score = 0.2
                        reasons.append("❌ Too tender")
                else:
                    hardiness_score = 0.5
        except:
            hardiness_score = 0.5
        
        score += hardiness_score * 0.4
        
        # Shade tolerance (compare to latitude/sun hours)
        shade = str(plant.get('shade', ''))
        if 'full sun' in shade.lower() or 'light shade' in shade.lower():
            shade_score = 1.0 if avg_temp_c > 10 else 0.8  # Sun-loving prefer warmer
        elif 'shade' in shade.lower():
            shade_score = 0.9 if precip_mm > 700 else 0.7  # Shade plants like moist
        else:
            shade_score = 0.8
        score += shade_score * 0.15
        
        # Moisture (precipitation match)
        moisture = str(plant.get('moisture', ''))
        if 'moist' in moisture.lower() or 'wet' in moisture.lower():
            moist_score = 1.0 if precip_mm > 700 else 0.6
        elif 'dry' in moisture.lower():
            moist_score = 1.0 if precip_mm < 600 else 0.7
        else:
            moist_score = 0.8
        score += moist_score * 0.15
        
        # ========== SOIL MATCHING (25%) ==========
        soil_pref = str(plant.get('soil', '')).lower()
        soil_type = location['soil_type'].lower()
        soil_ph = location['soil_ph']
        
        # Soil type match
        soil_keywords = {
            'loam': ['loam', 'garden', ''],
            'acidic': ['acid', 'peat'],
            'alkaline': ['chalk', 'lime'],
            'sandy': ['sand', 'light'],
            'clay': ['heavy', 'clay']
        }
        
        soil_match = 0
        for pref_type, keywords in soil_keywords.items():
            if any(k in soil_pref for k in keywords):
                if pref_type in soil_type or pref_type == '':
                    soil_match += 0.25
                    reasons.append("✅ Soil type match")
        
        # pH match (plants often specify tolerances)
        if 'acid' in soil_pref and soil_ph < 6.5:
            soil_match += 0.25
            reasons.append("✅ Acid soil match")
        elif 'alkal' in soil_pref and soil_ph > 7.0:
            soil_match += 0.25
            reasons.append("✅ Alkaline soil match")
        elif 6.0 <= soil_ph <= 7.5:
            soil_match += 0.2  # Neutral OK for most
        
        score += soil_match * 0.25
        
        # ========== PHYSICAL/HABITAT MATCH (20%) ==========
        habit = str(plant.get('habit', '')).lower()
        height = plant.get('height', '')
        growth = str(plant.get('growth', '')).lower()
        microclimate = location['microclimate'].lower()
        
        # Habit suitability
        if 'shrub' in habit or 'tree' in habit:
            if microclimate in ['plain', 'foothills']:
                habit_score = 0.9
            else:
                habit_score = 0.7  # Woody less ideal in mountains
        elif 'perennial' in habit or 'annual' in habit:
            habit_score = 1.0  # Herbaceous very flexible
        elif 'ground cover' in habit:
            habit_score = 0.95  # Great for slopes
        else:
            habit_score = 0.8
        
        # Growth rate (fast growers for quick establishment)
        if 'fast' in growth:
            growth_score = 1.0
        elif 'medium' in growth:
            growth_score = 0.8
        else:
            growth_score = 0.6
        
        physical_score = (habit_score + growth_score) / 2
        score += physical_score * 0.20
        
        # ========== USEFULNESS BONUS (10%) ==========
        edib = plant.get('edibility_rating', 0)
        med = plant.get('medicinal_rating', 0)
        other = plant.get('other_uses_rating', 0)
        
        usefulness = (edib + med + other) / 3.0 * 0.1  # Max +10%
        score += usefulness
        
        # Safety check
        hazards = str(plant.get('known_hazards', ''))
        if 'poison' in hazards.lower() or 'toxic' in hazards.lower():
            score *= 0.7  # Penalty for hazardous plants
            reasons.append("⚠️ Known hazards")
        
        # Only keep reasonable matches
        plant_id = plants_df.loc[idx, 'id'] if 'id' in plants_df.columns else (idx + 1)
        if score >= min_score:
            scores.append({
                'plant_id': plant_id,  # Assuming 1-based indexing
                'latin_name': plant['latin_name'],
                'common_name': plant['common_name'],
                'suitability_score': round(score, 3),
                'reasons': '; '.join(reasons[:5]),  # Top 5 reasons
                'hardiness': plant['hardiness'],
                'habit': plant['habit'],
                'soil': plant['soil'],
                'shade': plant['shade'],
                'moisture': plant['moisture'],
                'edibility': edib,
                'medicinal': med
            })
    
    # Rank and return top recommendations
    results_df = pd.DataFrame(scores).sort_values('suitability_score', ascending=False).head(top_n)
    
    # Save to recommendations table
    if not results_df.empty:
        results_df['location_id'] = location_id
        results_df[['location_id', 'plant_id', 'suitability_score', 'reasons']].to_sql('recommendations', conn, if_exists='append', index=False)
    
    conn.close()
    
    print(f"\n🎯 TOP {len(results_df)} RECOMMENDATIONS (score > {min_score}):")
    print(results_df[['latin_name', 'common_name', 'suitability_score', 'reasons']].round(3).to_string(index=False))
    
    return results_df

# ========== USAGE EXAMPLE ==========
if __name__ == "__main__":
    # After your setup code, run recommendations:
    sofia_recs = calculate_plant_suitability(
        location_id=1,  # Sofia ID from your setup
        lat=42.6977, 
        lon=23.3219,
        top_n=15,
        min_score=0.5  # Only good matches
    )
    
    if sofia_recs is not None:
        print(f"\n💾 Saved {len(sofia_recs)} recommendations to database")
        
        # Export to CSV for garden planning
        sofia_recs.to_csv('sofia_garden_recommendations.csv', index=False)
        print("📄 Exported to sofia_garden_recommendations.csv")


🌱 Calculating plant suitability scores...

🎯 TOP 15 RECOMMENDATIONS (score > 0.5):
                latin_name                                          common_name  suitability_score                                      reasons
             Urtica dioica                   Stinging Nettle, California nettle              1.379 ✅ Perfect hardiness match; ✅ Soil type match
       Althaea officinalis                     Marsh Mallow, Common marshmallow              1.346 ✅ Perfect hardiness match; ✅ Soil type match
   Pueraria montana lobata                                           Kudzu Vine              1.346 ✅ Perfect hardiness match; ✅ Soil type match
     Hippophae salicifolia                          Willow-Leaved Sea Buckthorn              1.316 ✅ Perfect hardiness match; ✅ Soil type match
       Crataegus laevigata Midland Hawthorn, Smooth hawthorn, English  Hawthorn              1.316 ✅ Perfect hardiness match; ✅ Soil type match
             Rumex acetosa                           

Vectorization

In [18]:
def vectorized_scoring(plants_df, garden):
    # Example with column names adapted to your data
    min_temp_ok = plants_df['MinTempC'] <= garden['max_temp']
    max_temp_ok = plants_df['MaxTempC'] >= garden['min_temp']
    
    temp_score = 0.4 * (min_temp_ok.astype(float) * max_temp_ok.astype(float))
    
    # Similarly create vectors for other factors
    precip_ok = plants_df['PrecipitationMinMM'] <= garden['precip']
    precip_score = 0.2 * precip_ok.astype(float)
    
    ph_ok = (plants_df['pHmin'] <= garden['soil_ph']) & (plants_df['pHmax'] >= garden['soil_ph'])
    ph_score = 0.15 * ph_ok.astype(float)
    
    hardiness_ok = plants_df['Hardiness'].str.contains(r'H[4-7]').fillna(False)
    hardiness_score = 0.15 * hardiness_ok.astype(float)
    
    altitude_ok = (garden['altitude'] >= plants_df['AltitudeMin']) & (garden['altitude'] <= plants_df['AltitudeMax'])
    altitude_score = 0.05 * altitude_ok.astype(float)
    
    microclimate_score = 0.05 * (plants_df['LightRequirement'] == garden['microclimate']).astype(float)
    
    total_score = temp_score + precip_score + ph_score + hardiness_score + altitude_score + microclimate_score
    
    plants_df['total_score'] = total_score.clip(upper=1.0)
    
    # Find reasons vectorized - more complex but possible with masks and apply if needed
    # For simplicity, skip reasons or describe them by filtering plants with score < threshold
    
    return plants_df.sort_values('total_score', ascending=False)


In [21]:
plants.columns.to_list()

['latin_name',
 'common_name',
 'habit',
 'height',
 'hardiness',
 'growth',
 'soil',
 'shade',
 'moisture',
 'edibility_rating',
 'medicinal_rating',
 'other_uses_rating',
 'family',
 'known_hazards',
 'habitats',
 'range',
 'weed_potential',
 'summary',
 'physical_characteristics',
 'synonyms',
 'habitats_section',
 'edible_uses',
 'medicinal_uses',
 'other_uses',
 'cultivation_details',
 'propagation',
 'other_names',
 'found_in',
 'weed_potential_section',
 'conservation_status',
 'expert_comment',
 'author',
 'botanical_references',
 'Hardiness']

In [23]:
location_id = add_user_location_real_data(42.6977, 23.3219, "My Sofia Garden")

conn = sqlite3.connect('garden_planner.db')
plants = pd.read_sql("SELECT *, Hardiness as Hardiness FROM plants", conn)
garden = pd.read_sql(f"""
    SELECT l.*, c.avg_temp, c.min_temp, c.max_temp, c.precip, c.frost_days
    FROM locations l
    JOIN climate_data c ON l.id = c.location_id 
    WHERE l.id = {location_id} AND c.scenario = 'historical_2015-2024'
""", conn).iloc[0].to_dict()

recommended_plants = vectorized_scoring(plants, garden)
top_plants = recommended_plants.head(20)
print(top_plants[['Common', 'Latin', 'total_score']])

🌍 Fetching real data for My Sofia Garden (42.6977, 23.3219)...
✅ Location created with REAL data:
   name: My Sofia Garden
   lat: 42.6977
   lon: 23.3219
   altitude: 556
   microclimate: hilly
   soil_type: neutral_loam
   soil_ph: 6.5
   geology: unknown
   subsurface_water: 2.0
   country: България
   region: Unknown
   city: София
📍 ID: 5


IndexError: single positional indexer is out-of-bounds